In [5]:
import pandas as pd

**Задание 1**  
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic  
- для источников paid и email из России - ставим ad  
- для источников paid и email не из России - ставим other  
- все остальные варианты берем из traffic_source без изменений  

In [6]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head(3)

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex


In [7]:
log['source_type'] = log.traffic_source   # для начала скопируем всё без изменения, а потом накатим конкретные фильтры

log.loc[log.traffic_source.isin(['yandex', 'google']), 'source_type'] = 'organic'
log.loc[(log.traffic_source.isin(['paid', 'email'])) & (log.region == 'Russia') , 'source_type'] = 'ad'
log.loc[(log.traffic_source.isin(['paid', 'email'])) & (log.region != 'Russia') , 'source_type'] = 'other'

log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


In [8]:
# проверка с помощью пивота
log.pivot_table(index = ['traffic_source', 'source_type'] , columns = 'region', values = 'url', aggfunc = 'count', fill_value = 0)

region                      Australia  Belarus  China  India  Russia  Ukraine
traffic_source source_type                                                   
direct         direct             222      998    397    134    4052     1210
email          ad                   0        0      0      0    2209        0
               other              115      569    252     82       0      648
google         organic             91      376    151     55    1435      399
paid           ad                   0        0      0      0    1581        0
               other               74      368    138     39       0      443
yandex         organic             88      446    171     59    1636      500

**Задание 2**  
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла в датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [25]:
import re

urls = pd.read_csv('urls.txt')
urls[urls.url.str.contains('/\d{8}-')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


**Задание 3**  
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [48]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})
data.head()

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [49]:
from pymystem3 import Mystem
m = Mystem()

In [51]:
data['lemmas'] = data['keyword'].apply(lambda x: ''.join(m.lemmatize(x)).replace('\n', ''))
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль
1,доллары в рубли,114173,доллар в рубль
2,100 долларов в рублях,97534,100 доллар в рубль
3,курс рубля,53546,курс рубль
